In [52]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))

In [54]:
# 1. Import necessary modules
from src.Agents.call_center.call_scripts import ScriptType
from src.Agents.call_center.state import CallStep, VerificationStatus
from src.Agents.call_center.utils import (
    prepare_script_parameters,
    create_system_prompt
)

In [55]:
from src.Agents.call_center.prompts import *

In [58]:
# 2. Define minimal test data
test_state = {
    'verified_client_name': 'John Smith',
    'current_call_step': CallStep.NAME_VERIFICATION.value,
    'name_verification_status': VerificationStatus.WRONG_PERSON.value
}

test_client_info = {
    'full_name': 'John Smith',
    'outstanding_amount': 'R 1,200',
    'subscription_amount': 'R 300',
}

test_script_type = ScriptType.RATIO_1_INFLOW



# Call function
system_prompt = create_system_prompt(
    test_state, 
    test_script_type, 
    test_client_info, 
)
print(system_prompt)

{'verified_client_name': 'John Smith', 'current_call_step': 'name_verification', 'name_verification_status': 'WRONG_PERSON'}

You are AI Agent, a professional debt collection agent for Cartrack making an OUTBOUND CALL to collect payment.

CURRENT CALL POSITION:
- Current Step: name_verification
- Overall Progress: 
   1. Introduction
-> 2. Name Verification (CURRENT)
   3. Details Verification
   4. Reason for Call
   5. Negotiation
   6. Promise to Pay
   7. Payment Processing
   8. Subscription Reminder
   9. Client Details Update
   10. Referrals
   11. Further Assistance
   12. Closing
- Client: John Smith
- Verification Status: INSUFFICIENT_INFO
- Outstanding Amount: R 1,200

CLIENT INFORMATION:
Client details not yet verified.

SCRIPT GUIDANCE:
May I speak to John Smith, please?


VERIFICATION GUIDANCE:
- Attempt 1 of 3
- Current status: WRONG_PERSON

RESPONSE APPROACH BY STATUS:
- INSUFFICIENT_INFO: 
  • Ask directly for identity confirmation
  • First attempt: Casual and friend

In [48]:
parameters = prepare_script_parameters(test_state, test_client_info,{})
parameters

{'agent_name': 'AI Agent',
 'client_full_name': 'John Smith',
 'client_name': 'John',
 'client_title': 'Mr./Ms.',
 'salutation': 'Sir/Madam',
 'outstanding_amount': 'R 1,200',
 'subscription_amount': 'R 300',
 'subscription_date': 'the usual date',
 'cancellation_fee': 'the cancellation fee',
 'total_balance': 'the total balance',
 'name_verification_status': 'INSUFFICIENT_INFO',
 'name_verification_attempts': 1,
 'max_name_verification_attempts': 3,
 'details_verification_status': 'INSUFFICIENT_INFO',
 'details_verification_attempts': 1,
 'max_details_verification_attempts': 3,
 'matched_fields': [],
 'ptp_amount_plus_fee': 'R 1,200 plus R10',
 'amount_with_fee': 'R 1,200 plus R10 fee',
 'current_step': 'introduction',
 'step_number': 1,
 'payment_secured': False,
 'call_outcome': 'UNKNOWN',
 'department': 'relevant',
 'priority_level': 'standard',
 'case_type': 'general',
 'ticket_number': '',
 'response_time': '24-48 hours',
 'additional_consequences': ''}

### Print Prompts

In [51]:
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 1. Define initial test state with all required fields
INITIAL_STATE = {
    # Identity and verification
    'verified_client_name': 'John Smith',
    'current_call_step': CallStep.INTRODUCTION.value,
    'name_verification_status': VerificationStatus.INSUFFICIENT_INFO.value,
    'name_verification_attempts': 1,
    'details_verification_attempts': 0,
    'matched_fields': [],  # Required for details verification
    'verification_items_needed': 3,
    'verification_items_matched': [],

    # Confirmation counters
    'third_party_confirmation_count': 1,
    'unavailable_confirmation_count': 1,
    'wrong_person_confirmation_count': 1,

    # Payment information
    'payment_secured': False,
    'payment_method': None,
    'agreed_amount': None,
    'payment_agreement_confidence': 0.0,

    # Call tracking
    'call_outcome': None,
    'previous_step': None,
    'is_call_ended': False,

    # Messages history
    'messages': [],

    # Escalation details
    'escalation_department': 'Client Services',
    'priority_level': 'standard',
    'case_type': 'payment',
    'ticket_number': 'CT123456',

    # Script type specific
    'script_type': ScriptType.RATIO_1_INFLOW.value,

    # Additional tracking
    'query_detected': False,
    'cancellation_requested': False,
    'escalation_requested': False
}

# Create test state from initial state
test_state = INITIAL_STATE.copy()

# Test client information
test_client_info = {
    'full_name': 'John Smith',
    'outstanding_amount': 'R 1,200',
    'subscription_amount': 'R 300',
    'subscription_date': 'the 15th of every month',
    'account_status': 'overdue',
    'cancellation_fee': 'R 3,600',
    'total_balance': 'R 4,800',
    'email': 'john.smith@email.com',
    'phone': '0123456789',
    'contact': {'mobile': '0123456789'},
    'vehicles': [
        {
            'make': 'Toyota',
            'model': 'Corolla',
            'registration': 'ABC123GP',
            'color': 'Silver'
        }
    ],
    'payment_history': [
        {'date': '2025-03-15', 'amount': 'R 300', 'status': 'Successful'},
        {'date': '2025-02-15', 'amount': 'R 300', 'status': 'Failed'},
        {'date': '2025-01-15', 'amount': 'R 300', 'status': 'Successful'}
    ]
}

test_agent_name = "AI Agent"
test_script_type = ScriptType.RATIO_1_INFLOW

# 2. Define steps to test
call_steps = [
    CallStep.INTRODUCTION.value,
    CallStep.NAME_VERIFICATION.value,
    CallStep.DETAILS_VERIFICATION.value,
    CallStep.REASON_FOR_CALL.value,
    CallStep.NEGOTIATION.value,
    CallStep.PROMISE_TO_PAY.value,
    CallStep.DEBICHECK_SETUP.value,
    CallStep.SUBSCRIPTION_REMINDER.value,
    CallStep.PAYMENT_PORTAL.value,
    CallStep.CLIENT_DETAILS_UPDATE.value,
    CallStep.REFERRALS.value,
    CallStep.FURTHER_ASSISTANCE.value,
    CallStep.CANCELLATION.value,
    CallStep.ESCALATION.value,
    CallStep.CLOSING.value,
    CallStep.QUERY_RESOLUTION.value
]

# 3. Define special cases
special_cases = [
    "third_party",
    "not_available",
    "wrong_person",
    "failed_verification"
]

# 4. Script type test cases
script_type_tests = [
    (ScriptType.RATIO_2_3_INFLOW, CallStep.NEGOTIATION.value, "RATIO 2&3 NEGOTIATION"),
    (ScriptType.PRE_LEGAL_120_PLUS, CallStep.REASON_FOR_CALL.value, "PRE-LEGAL REASON FOR CALL"),
    (ScriptType.PRE_LEGAL_150_PLUS, CallStep.INTRODUCTION.value, "ATTORNEY INTRODUCTION"),
    (ScriptType.RECENTLY_SUSPENDED_120_PLUS, CallStep.NEGOTIATION.value, "SUSPENDED NEGOTIATION")
]

def print_prompt_header(title: str) -> None:
    """Print formatted header for prompt section"""
    print(f"\n{'='*80}")
    print(f"SYSTEM PROMPT FOR: {title}")
    print(f"{'='*80}\n")

def print_prompt_footer(title: str) -> None:
    """Print formatted footer for prompt section"""
    print(f"\n{'='*80}")
    print(f"END OF PROMPT FOR: {title}")
    print(f"{'='*80}\n")

def reset_test_state() -> None:
    """Reset test state to initial values"""
    global test_state
    test_state = INITIAL_STATE.copy()

def set_state_for_step(step: str) -> None:
    """Set appropriate state for a given step"""
    verification_steps = {
        CallStep.INTRODUCTION.value,
        CallStep.NAME_VERIFICATION.value,
        CallStep.DETAILS_VERIFICATION.value
    }
    
    # Update basic step info
    test_state['current_call_step'] = step
    test_state['previous_step'] = test_state['current_call_step']
    
    # Set verification status
    if step in verification_steps:
        test_state['name_verification_status'] = VerificationStatus.INSUFFICIENT_INFO.value
    else:
        test_state['name_verification_status'] = VerificationStatus.VERIFIED.value
        test_state['verified_client_name'] = test_client_info['full_name']
    
    # Set step-specific state
    if step == CallStep.PROMISE_TO_PAY.value:
        test_state['payment_method'] = PaymentMethod.IMMEDIATE_DEBIT.value
        test_state['agreed_amount'] = test_client_info['outstanding_amount']
    elif step == CallStep.DEBICHECK_SETUP.value:
        test_state['payment_method'] = PaymentMethod.DEBICHECK.value
    elif step == CallStep.CLOSING.value:
        test_state['payment_secured'] = True
        test_state['call_outcome'] = 'PAYMENT_ARRANGED'
    elif step == CallStep.ESCALATION.value:
        test_state['priority_level'] = 'urgent'
        test_state['escalation_requested'] = True

def print_all_system_prompts() -> None:
    """Generate and print system prompts for all test cases"""
    logger.info("Starting system prompt generation tests")
    
    # Test standard call steps
    print("Generating system prompts for all call steps...\n")
    
    for step in call_steps:
        print_prompt_header(step)
        reset_test_state()
        set_state_for_step(step)
        
        # Generate and print prompt
        system_prompt = create_system_prompt(
            test_state,
            test_script_type,
            test_client_info,
            test_agent_name
        )
        print(system_prompt)
        print_prompt_footer(step)
    
    # Test special cases
    print("\nGenerating system prompts for special cases...\n")
    
    verification_status_map = {
        "third_party": VerificationStatus.THIRD_PARTY.value,
        "not_available": VerificationStatus.UNAVAILABLE.value,
        "wrong_person": VerificationStatus.WRONG_PERSON.value,
        "failed_verification": VerificationStatus.VERIFICATION_FAILED.value
    }
    
    for case in special_cases:
        print_prompt_header(f"SPECIAL CASE: {case}")
        reset_test_state()
        
        # Set special case state
        test_state.update({
            'current_call_step': CallStep.NAME_VERIFICATION.value,
            'name_verification_status': verification_status_map[case],
            'name_verification_attempts': 3 if case == "failed_verification" else 1
        })
        
        # Generate and print prompt
        system_prompt = create_system_prompt(
            test_state,
            test_script_type,
            test_client_info,
            test_agent_name
        )
        print(system_prompt)
        print_prompt_footer(f"SPECIAL CASE: {case}")
    
    # Test different script types
    print("\nGenerating system prompts for different script types...\n")
    
    for script_type, step, label in script_type_tests:
        print_prompt_header(label)
        reset_test_state()
        set_state_for_step(step)
        
        # Generate and print prompt
        system_prompt = create_system_prompt(
            test_state,
            script_type,
            test_client_info,
            test_agent_name
        )
        print(system_prompt)
        print_prompt_footer(label)
    
    logger.info("Completed system prompt generation tests")

if __name__ == "__main__":
    print_all_system_prompts()

2025-04-23 13:12:40,321 - INFO - Starting system prompt generation tests
2025-04-23 13:12:40,321 - WARNING - Section 'DETAILS_VERIFICATION' not found or invalid for script type 'ratio_1_inflow'
2025-04-23 13:12:40,322 - WARNING - Section 'ESCALATION' not found or invalid for script type 'ratio_1_inflow'
2025-04-23 13:12:40,323 - WARNING - Section 'CLOSING' not found or invalid for script type 'ratio_1_inflow'
2025-04-23 13:12:40,324 - INFO - Completed system prompt generation tests


Generating system prompts for all call steps...


SYSTEM PROMPT FOR: introduction


You are AI Agent, a professional debt collection agent for Cartrack making an OUTBOUND CALL to collect payment.

CURRENT CALL POSITION:
- Current Step: introduction
- Overall Progress: 
-> 1. Introduction (CURRENT)
   2. Name Verification
   3. Details Verification
   4. Reason for Call
   5. Negotiation
   6. Promise to Pay
   7. Payment Processing
   8. Subscription Reminder
   9. Client Details Update
   10. Referrals
   11. Further Assistance
   12. Closing
- Client: John Smith
- Verification Status: INSUFFICIENT_INFO
- Outstanding Amount: R 1,200

CLIENT INFORMATION:
Client details not yet verified.

SCRIPT GUIDANCE:
Good day, you are speaking to AI Agent calling from the Cartrack Accounts Department. May I speak to John Smith?


INTRODUCTION PHASE GUIDANCE:
- Be concise and professional - remember this is an outbound call
- Begin with a courteous greeting ("Good day")
- Identify yourself by name, 